In [4]:
import json
import pandas as pd

In [5]:
data = pd.read_json('sl_self_service_cleaned.json', orient='records',lines=True)
data["label"].value_counts().head(50)

no     6447
yes     859
Name: label, dtype: int64